In [ ]:
!pip install datasets transformers jsonlines

In [2]:
from huggingface_hub import interpreter_login

interpreter_login()

/users/klee12/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token is valid (permission: read).
Your token has been saved to /users/klee12/.cache/huggingface/token
Login successful


In [3]:
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer

# Load the dataset
dataset = load_dataset("andstor/smart_contract_code_comments", 'default')

# Print basic information about the dataset
print(dataset)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Generating train split: 1267441 examples [00:38, 32770.11 examples/s]
Generating validation split: 130849 examples [00:04, 29320.21 examples/s]
Generating test split: 143080 examples [00:04, 29848.65 examples/s]

DatasetDict({
    train: Dataset({
        features: ['contract_name', 'file_path', 'contract_address', 'language', 'class_name', 'class_code', 'class_documentation', 'class_documentation_type', 'func_name', 'func_code', 'func_documentation', 'func_documentation_type', 'compiler_version', 'license_type', 'swarm_source', 'meta', '__index_level_0__'],
        num_rows: 1267441
    })
    validation: Dataset({
        features: ['contract_name', 'file_path', 'contract_address', 'language', 'class_name', 'class_code', 'class_documentation', 'class_documentation_type', 'func_name', 'func_code', 'func_documentation', 'func_documentation_type', 'compiler_version', 'license_type', 'swarm_source', 'meta', '__index_level_0__'],
        num_rows: 130849
    })
    test: Dataset({
        features: ['contract_name', 'file_path', 'contract_address', 'language', 'class_name', 'class_code', 'class_documentation', 'class_documentation_type', 'func_name', 'func_code', 'func_documentation', 'func_docume

In [4]:
def clean_code(dataset):
    remove_cols = ['file_path', 'contract_address', 'language', 'class_name', 'class_documentation_type', 'func_name', 'func_code', 'func_documentation', 'func_documentation_type', 'compiler_version', 'license_type', 'swarm_source', 'meta', '__index_level_0__']
    code = dataset.remove_columns(remove_cols)
    
    full_code = pd.DataFrame(code)
    full_code.drop_duplicates(inplace=True)
    
    return full_code

In [5]:
import re

# Define the function that extracts function headers
def extract_function_header(func_code):
    # Regular expression to match Solidity function declarations
    func_header_regex = r'function\s*[\w\d_]*\s*\([^)]*\)\s*[^{]*'
    # Find all matches in the func_code
    matches = re.findall(func_header_regex, func_code)
    # Assume each func_code contains one function and take the first match
    # If there are multiple matches, this will need adjustment
    return matches[0] if matches else None

In [6]:
def clean_func(dataset):
    remove_func_col = ['file_path', 'contract_address', 'class_code', 'class_documentation', 'language', 'class_name', 'class_documentation_type', 'func_documentation_type', 'compiler_version', 'license_type', 'swarm_source', 'meta', '__index_level_0__']
    func_code = dataset.remove_columns(remove_func_col)
    
    func = pd.DataFrame(func_code)
    
    func['func_header'] = func['func_code'].apply(extract_function_header)

    # filter out rows with no function headers
    func = func[func['func_header'].apply(lambda x: x is not None and len(x) > 0)]
    
    # group by contract
    grouped = func.groupby('contract_name').agg({
        'func_header': list,
        'func_documentation': list
    }).reset_index()
    
    return grouped

In [7]:
train = dataset['train']
train_code = clean_code(train)
train_func = clean_func(train)

train_clean = pd.merge(train_code, train_func, on='contract_name', how='left')
train_clean = train_clean.drop_duplicates(subset=['contract_name'], keep='first')

In [8]:
subset_filter = train_clean[~train_clean['class_code'].str.startswith('interface')]
subset_filter = subset_filter[~subset_filter['class_code'].str.startswith('contract Token')]
subset_filter = subset_filter[~subset_filter['class_code'].str.startswith('contract Owned')]